<a href="https://colab.research.google.com/github/michaeledge27/CSCI290/blob/main/notebooks/NaiveBayesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from scipy.stats import norm
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [15]:
penguins = pd.read_csv("https://github.com/mbrudd/csci290/raw/refs/heads/main/data/penguins.csv")

In [17]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 7   year               344 non-null    int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 21.6+ KB


In [19]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [3]:
priors = penguins["species"].value_counts( normalize=True )
priors

,proportion
species,
Adelie,0.441860
Gentoo,0.360465
Chinstrap,0.197674


In [10]:
def calculate_priors(y):
    return y.value_counts( normalize=True ).to_dict()

In [6]:
def calculate_likelihoods(X, y):
  likelihoods = {}
  for feature in X.columns:
    likelihoods[feature] = {}
    for clas in y.unique():
      subset = X[y == clas]
      value_counts = subset[feature].value_counts(normalize=True)
      likelihoods[feature][clas] = value_counts.to_dict()
  return likelihoods


In [8]:
def predict_naive_bayes(instance, priors, likelihoods, classes):
    posteriors = {}

    for clas in classes:
      posterior = np.log(priors[clas])
      for feature, value in instance.items():
        if value in likelihoods.get(feature, {}).get(clas, {}):
            posterior += np.log(likelihoods[feature][clas].get(value))

    posteriors[clas] = posterior

    return max(posteriors, key=posteriors.get)

In [9]:
def naive_bayes(X, y, new_instance):
  priors = calculate_priors(y)
  likelihoods = calculate_likelihoods(X, y)
  classes = y.unique()
  return predict_naive_bayes(new_instance, priors, likelihoods, classes)

In [11]:
X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
y = penguins['species']

In [12]:
new_penguin_instance = {
    'bill_length_mm': 45.0,
    'bill_depth_mm': 14.0,
    'flipper_length_mm': 210.0,
    'body_mass_g': 4500
}

In [13]:
predicted_species = naive_bayes(X, y, new_penguin_instance)
print(f"Predicted species: {predicted_species}")

Predicted species: Chinstrap
